In [ ]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from IPython.display import display


In [ ]:
%load_ext autoreload
%autoreload 2


### Load ACS data:


In [ ]:
CENSUS_DATA_ROOT = './data/clean_census_data/'


In [ ]:
# Import total population:
census_population = pd.read_csv(CENSUS_DATA_ROOT+"population_totals.csv")

## Import employment data:
#census_employment = pd.read_csv(CENSUS_DATA_ROOT+"employment_population_ratio.csv")

# Import geography data:
census_geography = pd.read_csv(CENSUS_DATA_ROOT+"county_geometries.csv")

# Import employment data:
census_employment = pd.read_csv(CENSUS_DATA_ROOT+"employment_status.csv")

# Import geography data:
census_age_total = pd.read_csv(CENSUS_DATA_ROOT+"age_information_total.csv")

# Import urban/rural data:
census_urban_rural = pd.read_csv(CENSUS_DATA_ROOT+"urban_rural_by_county.csv")

# Import income data:
census_income = pd.read_csv(CENSUS_DATA_ROOT+"income_totals_household.csv")

# Add identifiers to total population:
census_population['Geographic Area Name'] = census_population['CTYNAME']+', '+census_population['STNAME']
census_population = census_population.groupby(['Geographic Area Name','CTYNAME','STNAME']).sum().reset_index()
census_population = census_employment[['id','Geographic Area Name']].drop_duplicates().merge(
    census_population, left_on=['Geographic Area Name'], right_on=['Geographic Area Name'], how='outer',
)
census_population.head()


In [ ]:
# Get geography data:
data = census_geography[['AFFGEOID','ALAND']].rename(columns={
    'AFFGEOID' : 'id',
    'ALAND' : 'area_square_meters',
})
# Get population data:
data = data.merge(census_population, left_on=['id'], right_on=['id'])
# Get employment data:
data = data.merge(census_employment,left_on=['id','Geographic Area Name'], right_on=['id','Geographic Area Name'])
# Get urban/rural data:
data = data.merge(
    census_urban_rural.rename(columns={'2013 code' : 'code_2013'})[['id','code_2013','type_2013']],
    left_on=['id'], right_on=['id'],
)
# Compute metrics:
data['pop_density'] = data['TOT_POP']/(data['area_square_meters']/1e6)
# Rename columns:
data = data.rename(columns={'id':'county_id'})

data
